In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
from lxml import etree
from typing import List, Tuple

def parse(file_name, texts, labels):
  with open(file_name, mode='rb') as fp:
    xml_data = fp.read()
  root = etree.fromstring(xml_data)
  for database in root.getchildren():
    if database.tag == 'database':
      for table in database.getchildren():
        if table.tag != 'table':
          continue
        new_text = None
        new_label = None
        for column in table.getchildren():
          if column.get('name') == 'text':
            new_text = str(column.text).strip()
            if new_label is not None:
              break
          elif column.get('name') not in {'id', 'twitid', 'date'}:
            if new_label is None:
              label_candidate = str(column.text).strip()
              if label_candidate == '-1':
                new_label = 0
              elif label_candidate == '0':
                new_label = 1
              elif label_candidate == '1':
                new_label = 2
                if new_text is not None:
                  break
        if (new_text is None) or (new_label is None):
          raise ValueError('File `{0}` contains some error!'.format(file_name))
        texts.append(new_text)
        labels.append(new_label)
      break

In [ ]:
def load_sentirueval_2016(file_name1: str, file_name2: str) -> Tuple[List[str], List[str]]:
  texts = []
  labels = []
  parse(file_name1, texts, labels)
  parse(file_name2, texts, labels)
  return texts, labels

In [ ]:
texts, labels = load_sentirueval_2016('drive/MyDrive/bank_train_2016.xml', 'drive/MyDrive/tkk_train_2016.xml')
texts_test, labels_test = load_sentirueval_2016('drive/MyDrive/banks_test_etalon.xml', 'drive/MyDrive/tkk_test_etalon.xml')

In [ ]:
!rm -f ft_native_300_ru_twitter_nltk_word_tokenize.bin
!wget http://files.deeppavlov.ai/embeddings/ft_native_300_ru_twitter_nltk_word_tokenize.bin

--2022-01-03 16:12:06--  http://files.deeppavlov.ai/embeddings/ft_native_300_ru_twitter_nltk_word_tokenize.bin
Resolving files.deeppavlov.ai (files.deeppavlov.ai)... 178.63.27.41
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|178.63.27.41|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.deeppavlov.ai/embeddings/ft_native_300_ru_twitter_nltk_word_tokenize.bin [following]
--2022-01-03 16:12:07--  https://files.deeppavlov.ai/embeddings/ft_native_300_ru_twitter_nltk_word_tokenize.bin
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|178.63.27.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3417475450 (3.2G) [application/octet-stream]
Saving to: ‘ft_native_300_ru_twitter_nltk_word_tokenize.bin’

ft_native_300_ru_tw 100%[===================>]   3.18G  9.76MB/s    in 5m 31s  

2022-01-03 16:17:39 (9.86 MB/s) - ‘ft_native_300_ru_twitter_nltk_word_tokenize.bin’ saved [3417475450/3417475450]



In [ ]:
import gensim
from gensim.models.fasttext import FastText

fasttext_model = FastText()
fasttext_model.file_name = 'ft_native_300_ru_twitter_nltk_word_tokenize.bin'
fasttext_model.load_binary_data()

In [ ]:
import numpy as np

def get_matrix(texts):
  output = []
  for text in texts:
    try:
      output.append(fasttext_model.wv[text])
    except KeyError:
      output.append(np.zeros(100))
  output = np.array(output, dtype=np.float32)
  output = output.reshape(len(texts), 100, 1, 1)
  return output

x_train, y_train  = get_matrix(texts), np.array(labels)
x_test, y_test = get_matrix(texts_test), np.array(labels_test)
print(x_train.shape, x_test.shape)

(18035, 100, 1, 1) (5560, 100, 1, 1)


In [ ]:
import random
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=RANDOM_SEED)

Сверточная нейронная сеть

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Reshape
from tensorflow.keras.initializers import he_uniform, glorot_uniform
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D, SpatialDropout2D

cnn = Sequential()
cnn.add(Conv2D(32, (2, 1), padding='same', activation='relu', input_shape=x_train.shape[1:],
               kernel_initializer=he_uniform(seed=RANDOM_SEED), name='Conv_Block1_Layer1'))
cnn.add(Conv2D(32, (2, 1), activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED),
               name='Conv_Block1_Layer2'))
cnn.add(MaxPooling2D(pool_size=(2, 1), name='MaxPool1'))
cnn.add(SpatialDropout2D(rate=0.15, name='SpatialDropout1', seed=RANDOM_SEED))

cnn.add(Conv2D(64, (2, 1), padding='same', activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED),
              name='Conv_Block2_Layer1'))
cnn.add(Conv2D(64, (2, 1), activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED),
               name='Conv_Block2_Layer2'))
cnn.add(MaxPooling2D(pool_size=(2, 1), name='MaxPool2'))
cnn.add(SpatialDropout2D(rate=0.15, name='SpatialDropout2', seed=RANDOM_SEED))
cnn.add(Flatten())
cnn.add(Dense(512, activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED), name='HiddenLayer'))
cnn.add(Dropout(rate=0.5, seed=RANDOM_SEED, name='DropoutAfterHidden'))
cnn.add(Dense(10, activation='softmax', kernel_initializer=glorot_uniform(seed=RANDOM_SEED), name='OutputLayer'))
cnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
cnn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv_Block1_Layer1 (Conv2D)  (None, 100, 1, 32)       96        
                                                                 
 Conv_Block1_Layer2 (Conv2D)  (None, 99, 1, 32)        2080      
                                                                 
 MaxPool1 (MaxPooling2D)     (None, 49, 1, 32)         0         
                                                                 
 SpatialDropout1 (SpatialDro  (None, 49, 1, 32)        0         
 pout2D)                                                         
                                                                 
 Conv_Block2_Layer1 (Conv2D)  (None, 49, 1, 64)        4160      
                                                                 
 Conv_Block2_Layer2 (Conv2D)  (None, 48, 1, 64)        8256      
                                                      

In [ ]:
BATCH_SIZE = 128

cnn.fit(
    x_train, y_train,
    batch_size=BATCH_SIZE,
    validation_data=(x_val, y_val),
    shuffle=True, epochs=100,
    callbacks=[
        EarlyStopping(
            monitor='val_loss', patience=5, restore_best_weights=True, verbose=1
        )
    ],
    verbose=1
)

Epoch 1/100
127/127 [==============================] - 2s 11ms/step - loss: 0.8297 - sparse_categorical_accuracy: 0.6469 - val_loss: 0.7474 - val_sparse_categorical_accuracy: 0.6508
Epoch 2/100
127/127 [==============================] - 1s 11ms/step - loss: 0.7377 - sparse_categorical_accuracy: 0.6703 - val_loss: 0.7328 - val_sparse_categorical_accuracy: 0.6552
Epoch 3/100
127/127 [==============================] - 2s 12ms/step - loss: 0.7240 - sparse_categorical_accuracy: 0.6807 - val_loss: 0.7104 - val_sparse_categorical_accuracy: 0.6813
Epoch 4/100
127/127 [==============================] - 1s 11ms/step - loss: 0.7133 - sparse_categorical_accuracy: 0.6859 - val_loss: 0.6972 - val_sparse_categorical_accuracy: 0.6935
Epoch 5/100
127/127 [==============================] - 1s 10ms/step - loss: 0.7021 - sparse_categorical_accuracy: 0.6961 - val_loss: 0.6870 - val_sparse_categorical_accuracy: 0.7068
Epoch 6/100
127/127 [==============================] - 1s 9ms/step - loss: 0.6942 - sparse

In [ ]:
y_pred = np.argmax(cnn.predict(x_test, batch_size=128), axis=-1)

In [ ]:
label_names = ['negative', 'neutral', 'positive']

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, target_names=label_names, digits=4))

              precision    recall  f1-score   support

    negative     0.5306    0.4237    0.4711      1801
     neutral     0.6558    0.8258    0.7310      3250
    positive     0.2414    0.0138    0.0260       509

    accuracy                         0.6212      5560
   macro avg     0.4759    0.4211    0.4094      5560
weighted avg     0.5773    0.6212    0.5823      5560



Сверточная нейронная сеть, обученная на аугментированных данных

In [ ]:
!pip install textattack[tensorflow,optional]

In [ ]:
from textattack.transformations import WordSwapRandomCharacterDeletion
from textattack.constraints.pre_transformation import RepeatModification
from textattack.augmentation import Augmenter


def augment_texts(texts, labels):
  transformation = WordSwapRandomCharacterDeletion(skip_first_char=True, skip_last_char=True)
  constraints = [RepeatModification()]
  aug = Augmenter(transformation=transformation, pct_words_to_swap=0.5, transformations_per_example=7)
  aug_texts = []
  aug_labels = []
  texts_total = len(texts)
  for text, label in zip(texts, labels):
    aug_text_options = aug.augment(text)
    aug_texts.extend(aug_text_options)
    aug_labels.extend([label] * len(aug_text_options))
    n += 1
    texts_total = len(texts)
  return aug_texts, aug_labels

In [ ]:
augmented_texts, augmented_labels = augment_texts(texts, labels)

In [ ]:
len(augmented_texts)

126174

In [ ]:
x_train, y_train  = get_matrix(augmented_texts), np.array(augmented_labels)
print(x_train.shape, x_test.shape)

(126174, 100, 1, 1) (5560, 100, 1, 1)


In [ ]:
cnn_with_augmentation = Sequential()
cnn_with_augmentation.add(Conv2D(32, (2, 1), padding='same', activation='relu', input_shape=x_train.shape[1:],
                                 kernel_initializer=he_uniform(seed=RANDOM_SEED), name='Conv_Block1_Layer1'))
cnn_with_augmentation.add(Conv2D(32, (2, 1), activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED),
                                 name='Conv_Block1_Layer2'))
cnn_with_augmentation.add(MaxPooling2D(pool_size=(2, 1), name='MaxPool1'))
cnn_with_augmentation.add(SpatialDropout2D(rate=0.15, name='SpatialDropout1', seed=RANDOM_SEED))
cnn_with_augmentation.add(Conv2D(64, (2, 1), padding='same', activation='relu',
                                 kernel_initializer=he_uniform(seed=RANDOM_SEED), name='Conv_Block2_Layer1'))
cnn_with_augmentation.add(Conv2D(64, (2, 1), activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED),
                                 name='Conv_Block2_Layer2'))
cnn_with_augmentation.add(MaxPooling2D(pool_size=(2, 1), name='MaxPool2'))
cnn_with_augmentation.add(SpatialDropout2D(rate=0.15, name='SpatialDropout2', seed=RANDOM_SEED))
cnn_with_augmentation.add(Flatten())
cnn_with_augmentation.add(Dense(512, activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED),
                                name='HiddenLayer'))
cnn_with_augmentation.add(Dropout(rate=0.5, seed=RANDOM_SEED, name='DropoutAfterHidden'))
cnn_with_augmentation.add(Dense(10, activation='softmax', kernel_initializer=glorot_uniform(seed=RANDOM_SEED),
                                name='OutputLayer'))
cnn_with_augmentation.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
cnn_with_augmentation.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv_Block1_Layer1 (Conv2D)  (None, 100, 1, 32)       96        
                                                                 
 Conv_Block1_Layer2 (Conv2D)  (None, 99, 1, 32)        2080      
                                                                 
 MaxPool1 (MaxPooling2D)     (None, 49, 1, 32)         0         
                                                                 
 SpatialDropout1 (SpatialDro  (None, 49, 1, 32)        0         
 pout2D)                                                         
                                                                 
 Conv_Block2_Layer1 (Conv2D)  (None, 49, 1, 64)        4160      
                                                                 
 Conv_Block2_Layer2 (Conv2D)  (None, 48, 1, 64)        8256      
                                                      

In [ ]:
BATCH_SIZE = 128

cnn_with_augmentation.fit(
    x_train, y_train,
    batch_size=BATCH_SIZE,
    validation_data=(x_val, y_val),
    shuffle=True, epochs=100,
    callbacks=[
        EarlyStopping(
            monitor='val_loss', patience=5, restore_best_weights=True, verbose=1
        )
    ],
    verbose=1
)

Epoch 1/100
888/888 [==============================] - 9s 9ms/step - loss: 0.7401 - sparse_categorical_accuracy: 0.6726 - val_loss: 0.7013 - val_sparse_categorical_accuracy: 0.6856
Epoch 2/100
888/888 [==============================] - 7s 8ms/step - loss: 0.7040 - sparse_categorical_accuracy: 0.6919 - val_loss: 0.6829 - val_sparse_categorical_accuracy: 0.6993
Epoch 3/100
888/888 [==============================] - 7s 8ms/step - loss: 0.6887 - sparse_categorical_accuracy: 0.6991 - val_loss: 0.6777 - val_sparse_categorical_accuracy: 0.6983
Epoch 4/100
888/888 [==============================] - 8s 9ms/step - loss: 0.6774 - sparse_categorical_accuracy: 0.7037 - val_loss: 0.6625 - val_sparse_categorical_accuracy: 0.7044
Epoch 5/100
888/888 [==============================] - 8s 9ms/step - loss: 0.6668 - sparse_categorical_accuracy: 0.7085 - val_loss: 0.6536 - val_sparse_categorical_accuracy: 0.7073
Epoch 6/100
888/888 [==============================] - 7s 8ms/step - loss: 0.6576 - sparse_cate

In [ ]:
y_pred = np.argmax(cnn_with_augmentation.predict(x_test, batch_size=128), axis=-1)

In [ ]:
print(classification_report(y_test, y_pred, target_names=label_names, digits=4))

              precision    recall  f1-score   support

    negative     0.4498    0.4119    0.4300     12604
     neutral     0.6432    0.7141    0.6768     22741
    positive     0.1408    0.0836    0.1049      3563

    accuracy                         0.5585     38908
   macro avg     0.4112    0.4032    0.4039     38908
weighted avg     0.5345    0.5585    0.5445     38908



Свёрточная нейронная сеть, как с применением аугментации (f1-мера для негативных, нейтральных и положительных комментариев соответственно: 43, 67, 10), так и без неё (47, 73, 02), даёт результаты хуже, чем логистическая регрессия (65, 78, 31). Аугментация ухудшила качество распознавания негативных и нейтральных комментариев и улучшила качество распознавания положительных.
